In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,18951.20,-0.13,-0.01,0.09,0.00,-0.02,0.00,-0.0,-0.00
3,-0.13,4388.74,-0.03,0.01,-0.02,0.00,0.01,0.0,0.00
4,-0.01,-0.03,1007.15,0.02,-0.00,0.00,-0.00,-0.0,0.00
5,0.09,0.01,0.02,229.11,0.00,-0.00,0.00,0.0,0.00
6,0.00,-0.02,-0.00,0.00,50.96,-0.00,-0.00,0.0,0.00
7,-0.02,0.00,0.00,-0.00,-0.00,11.33,0.00,0.0,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.53,0.0,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.6,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.0,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00003,0.00001,-0.00002,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00007,0.00000,0.00002
4,-0.00000,-0.00001,1.00000,0.00004,-0.00000,0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00000,-0.00003,0.00007,0.00001,0.00002
6,0.00000,-0.00003,-0.00000,0.00000,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00001,0.00000,-0.00003,-0.00001,1.00000,0.00002,0.00000,0.00001
8,0.00001,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00003,0.00001
9,-0.00002,0.00000,-0.00001,0.00001,0.00003,0.00000,0.00003,1.00000,-0.00001
10,-0.00000,0.00002,0.00002,0.00002,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.23045358940178046

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.99425016e+07 2.10735583e+06 1.08726005e+05 5.47136154e+03
 2.60072290e+02 1.20702265e+01 5.40539710e-01 2.36093185e-02
 6.71599581e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999511,-0.030540,-0.006606,-0.001493,-0.000333,-0.000075,-0.000016,-0.000004,-9.755497e-07
1,0.030281,0.998907,-0.034764,-0.007451,-0.001657,-0.000367,-0.000080,-0.000019,-4.672891e-06
2,0.007582,0.034207,0.998542,-0.040150,-0.008423,-0.001867,-0.000420,-0.000099,-2.392724e-05
3,0.002000,0.008673,0.039401,0.998025,-0.047023,-0.009869,-0.002193,-0.000521,-1.268781e-04
4,0.000533,0.002308,0.010060,0.045951,0.997083,-0.058696,-0.012358,-0.002918,-7.120365e-04
5,0.000148,0.000636,0.002774,0.012226,0.056955,0.995039,-0.078565,-0.017456,-4.242521e-03
6,0.000043,0.000185,0.000816,0.003569,0.016104,0.074815,0.988902,-0.124194,-2.790980e-02
7,0.000015,0.000063,0.000275,0.001204,0.005416,0.024407,0.112235,0.963472,-2.418641e-01
8,0.000008,0.000032,0.000141,0.000620,0.002789,0.012547,0.056092,0.236609,9.698991e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004894717737676801,
 0.0010925206869399906,
 0.0014581914893334602,
 0.001974616422848241,
 0.0029168818515464823,
 0.004960628420789703,
 0.011098129001348767,
 0.03652783030866702,
 0.030100892697244364]